In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import csv

output_file = open("amazon_data.csv","a")
url_data = pd.read_csv("amazon_raw_data.csv")
HEADERS = ({'User-Agent': 
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})
f = open('amazon_data.csv', 'w+', encoding="utf-8")
fieldnames = ['productTitle', 'productDescription', 'Rating', 'Price', 'Brand', 'Ingredients', 'About this item',
              'Frequently bought together', 'Products related to this item', 'URL']
writer = csv.DictWriter(f, fieldnames=fieldnames)

writer.writeheader()

## data cleaning

# list of attrs needed = ['productTitle', 'productDescription', 'Rating', 'Price', 'Brand', 'Ingredients', 'About this item',
#                           'Frequently bought together', 'Products related to this item']
url_data = url_data.dropna(subset=['URL'])
for i in range(len(url_data)):
    data = url_data.iloc[i]
    URL = data.URL
    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "lxml")

    # Extracting Title
    try:
        title = soup.find("span",attrs={"id":'productTitle'})
        print("at 1 title is", title)
        title = title.string.strip().replace(',', '')
        print("at 1 title is", title)
    #         writer.writerow({'productTitle': title})
    except:
        title = np.nan

    print("title:", title)

    # Extracting Product Descriptions
    try:
        desc = soup.find("div", attrs={"id":'productDescription'}).find("span").string
    #         writer.writerow({'productDescription': desc})
    except:
        desc = np.nan

    # Rating and Price
    rating = data.Rating

#     try:
#         rating = soup.find("span",attrs={"data-hook":'acr-average-starts-rating-text'})
#     except:
#         rating = np.nan

    price = data.Price

#     try:
#         price = soup.find("span",{"id":'snsDetailPagerPrice', "data-value": True})['data-value'] #.find("span",attrs={"id":'sns-base-price'}).string
#         print("price:",price)
#     except:
#         price = np.nan

    # No. of reviews
    try:
        reviews = soup.find("span",{"id":"acrCustomerReviewText"}).string
    #         writer.writerow({'productDescription': reviews})
    except:
        reviews = np.nan

    # Brand
    try:
        lis = soup.find("table", attrs={"class":"a-normal a-spacing-micro"}).find_all("span")
    #         print("lis ",lis)
        for idx, val in enumerate(lis):
            if(val.string=="Brand"):
                brand = lis[idx+1].string
                break
    #         writer.writerow({'Brand': brand})
        if brand is None:
            brand = np.nan
    except:
        brand = np.nan

    print("here, brand:",brand)
    # Ingredients
    try:
        flag=0
        lis = soup.find_all("div", attrs={"class" : "a-section content"})
        for idx, val in enumerate(lis):
            st_list = val.find_all(re.compile("h"))
            for idx2, val2 in enumerate(st_list):
                if(val2.string=="Ingredients"):
                    ingredients = lis[idx].text
                    flag=1
                    break
            if(flag):
                break
        if ingredients is None:
            ingredients = np.nan
    #         writer.writerow({'Ingredients': ingredients})
    except:
        ingredients = np.nan


    # About this item
    try:
        lis = soup.find("div", attrs={"id":"feature-bullets"}).find_all("li")
        features = []
        for val in lis:
            features.append(val.string)
    #         writer.writerow({'About this item': features})
    except:
        features = np.nan 

    # Frequently bought together
    # okay

    try:
        fbt = soup.find("div", attrs={"class": "cardRoot bucket"}).find_all("a", href=True)
        fbt_links = []
        for val in fbt:
            fbt_links.append('https://www.amazon.com/'+val['href'])
    #         writer.writerow({'Frequently bought together': fbt_links})
    except:
        fbt_links = np.nan

    # Product related to this item
    try:
        prt = soup.find("div", {"id":"sims-consolidated-2_feature_div"}).find_all("a", href=True)
        dic = {}
        for idx, val in enumerate(prt):
            try:
                temp = val['title']
                dic[temp] = 1
            except:
                temp=1

        prt_list_title = list(dic.keys())
    except:
        prt_list_title = np.nan
        
    writer.writerow({'productTitle': title, 'productDescription': desc, 'Rating': rating, 'Price': price, 'Brand': brand, 'Ingredients': ingredients, 'About this item': features, 'URL': URL})
    

csvreader = csv.reader(x.replace('nan', '') for x in f)
f.close()                            

at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        Eyeshadow Palette by Almay, Longlasting Eye Makeup, Primer Enriched with Antioxidant Vitamin E, Hypoallergenic, 010 Brown Eyes, 0.1 Oz       </span>
at 1 title is Eyeshadow Palette by Almay Longlasting Eye Makeup Primer Enriched with Antioxidant Vitamin E Hypoallergenic 010 Brown Eyes 0.1 Oz
title: Eyeshadow Palette by Almay Longlasting Eye Makeup Primer Enriched with Antioxidant Vitamin E Hypoallergenic 010 Brown Eyes 0.1 Oz
here, brand: Almay
at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        Maybelline New York Shadow Blocks Shadow Blocks Eyeshadow Palette, Stacked Eye Shadow Trio, 82nd &amp; Park Ave, 0.08 oz       </span>
at 1 title is Maybelline New York Shadow Blocks Shadow Blocks Eyeshadow Palette Stacked Eye Shadow Trio 82nd & Park Ave 0.08 oz
title: Maybelline New York Shadow Blocks Shadow Blocks Eyeshadow Palette Stacked Eye Shadow Trio 82nd & 

at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        Matte Eyeshadow Palette Pro 18 Colors Pigmented Shimmer Glitter Eye Shadow Palette Blendable Long Lasting Waterproof Makeup Cosmetics Halloween Makeup Kit (01# Seductress)       </span>
at 1 title is Matte Eyeshadow Palette Pro 18 Colors Pigmented Shimmer Glitter Eye Shadow Palette Blendable Long Lasting Waterproof Makeup Cosmetics Halloween Makeup Kit (01# Seductress)
title: Matte Eyeshadow Palette Pro 18 Colors Pigmented Shimmer Glitter Eye Shadow Palette Blendable Long Lasting Waterproof Makeup Cosmetics Halloween Makeup Kit (01# Seductress)
here, brand: Prism Makeup
at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        e.l.f. Liquid Metallic Eyeshadow, Gel Formula, Multi-Dimensional Finish For Bold Eye Looks, One-Swipe Coverage, Vegan &amp; Cruelty-Free, Moon, 0.1 Fl Oz       </span>
at 1 title is e.l.f. Liquid Metallic Eyeshadow Gel Formula Multi-Dimens

at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        e.l.f. Bite-Size Eyeshadows, Creamy, Blendable, Ultra-Pigmented &amp; Easy to Apply On-the-go, Long-lasting, Vegan &amp; Cruelty-Free, Rose Water, 0.12 Oz       </span>
at 1 title is e.l.f. Bite-Size Eyeshadows Creamy Blendable Ultra-Pigmented & Easy to Apply On-the-go Long-lasting Vegan & Cruelty-Free Rose Water 0.12 Oz
title: e.l.f. Bite-Size Eyeshadows Creamy Blendable Ultra-Pigmented & Easy to Apply On-the-go Long-lasting Vegan & Cruelty-Free Rose Water 0.12 Oz
here, brand: E.l.f.
at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        Revlon ColorStay Day to Night Eyeshadow Quad, Longwear Shadow Palette with Transitional Shades and Buttery Soft Feel, Crease &amp; Smudge Proof, 528 Passionate , 0.16 oz       </span>
at 1 title is Revlon ColorStay Day to Night Eyeshadow Quad Longwear Shadow Palette with Transitional Shades and Buttery Soft Feel Crease & Smu

at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        L.A. COLORS 16 Color Eyeshadow Palette, Haute, 1.02 Ounce       </span>
at 1 title is L.A. COLORS 16 Color Eyeshadow Palette Haute 1.02 Ounce
title: L.A. COLORS 16 Color Eyeshadow Palette Haute 1.02 Ounce
here, brand: L.A. COLORS
at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        10 Colors Eyeshadow Palette Smooth Matte Nude Eye Makeup Palette,High Pigmented, Naturing-Looking, Ultra-Blendable,Long Lasting High Neutral Eyeshadow Pawlette with 3 Eyeshadow Brush(Cement color)       </span>
at 1 title is 10 Colors Eyeshadow Palette Smooth Matte Nude Eye Makeup PaletteHigh Pigmented Naturing-Looking Ultra-BlendableLong Lasting High Neutral Eyeshadow Pawlette with 3 Eyeshadow Brush(Cement color)
title: 10 Colors Eyeshadow Palette Smooth Matte Nude Eye Makeup PaletteHigh Pigmented Naturing-Looking Ultra-BlendableLong Lasting High Neutral Eyeshadow Pawlette with 

at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        UCANBE Spotlight Eyeshadow Palette Professional 40 Color Eye Shadow Matte Shimmer Makeup Pallet Highly Pigmented Colorful Powder Long Lasting Waterproof Eye Shadow       </span>
at 1 title is UCANBE Spotlight Eyeshadow Palette Professional 40 Color Eye Shadow Matte Shimmer Makeup Pallet Highly Pigmented Colorful Powder Long Lasting Waterproof Eye Shadow
title: UCANBE Spotlight Eyeshadow Palette Professional 40 Color Eye Shadow Matte Shimmer Makeup Pallet Highly Pigmented Colorful Powder Long Lasting Waterproof Eye Shadow
here, brand: UCANBE
at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        Dark Brown EyeShadow Palette, Copper Eyeshadow glitter brown matte Shimmer Makeup Eyeshadow Pallet,Hazel Eyes Eyeshadow Palette ,smokey eye eyeshadow palette waterproof Long Lasting(C)       </span>
at 1 title is Dark Brown EyeShadow Palette Copper Eyeshadow glitter br

...

at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        Glow Recipe Plum Plump Hyaluronic Acid Face Cream - Dewy Glow Hydrating Face Moisturizer Plumps &amp; Balances Skin - Vegan Whipped Gel Moisturizer with Polyglutamic Acid &amp; Ice Willowherb (50ml)       </span>
at 1 title is Glow Recipe Plum Plump Hyaluronic Acid Face Cream - Dewy Glow Hydrating Face Moisturizer Plumps & Balances Skin - Vegan Whipped Gel Moisturizer with Polyglutamic Ac

at 1 title is None
title: nan
here, brand: nan
at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        Lipstick By Wet n Wild Mega Last High-Shine Lipstick Lip Color Makeup, Bright Pink Pinky Ring       </span>
at 1 title is Lipstick By Wet n Wild Mega Last High-Shine Lipstick Lip Color Makeup Bright Pink Pinky Ring
title: Lipstick By Wet n Wild Mega Last High-Shine Lipstick Lip Color Makeup Bright Pink Pinky Ring
here, brand: Wet n wild
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        CmaaDu 6Pcs Lipstick Matte Liquid Lipstick Lipgloss Set for Women labiales mate 24 horas originales m

at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        Lip Liner by Revlon, Colorstay Face Makeup with Built-in-Sharpener, Longwear Rich Lip Colors, Smooth Application, 630 Nude       </span>
at 1 title is Lip Liner by Revlon Colorstay Face Makeup with Built-in-Sharpener Longwear Rich Lip Colors Smooth Application 630 Nude
title: Lip Liner by Revlon Colorstay Face Makeup with Built-in-Sharpener Longwear Rich Lip Colors Smooth Application 630 Nude
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title:

at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        Triangle Powder Puff, Makeup Puff Velour Powder Puff, 6 Pcs Soft Powder Puffs for Face and Body Powder Cosmetic Foundation Sponge for Undereye Setting Powder Puff Makeup Tool (3 Black + 3 White)       </span>
at 1 title is Triangle Powder Puff Makeup Puff Velour Powder Puff 6 Pcs Soft Powder Puffs for Face and Body Powder Cosmetic Foundation Sponge for Undereye Setting Powder Puff Makeup Tool (3 Black + 3 White)
title: Triangle Powder Puff Makeup Puff Velour Powder Puff 6 Pcs Soft Powder Puffs for Face and Body Powder Cosmetic Foundation Sponge for Undereye Setting Powder Puff Makeup Tool (3 Black + 3 White)
here, brand: Norate
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, b

at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is None
title: nan
here, brand: nan
at 1 title is <span class="a-size-large product-title-word-break" id="productTitle">        Lunana Beauty 4 Point Eyebrow Pen, Micro Ink Brow Pen Waterproof Eyebrow Pencil with Micro-Fork Tips for Daily Natural Eye Brown Makeup (1# Chestnut)       </span>
at 1 title is Lunana Beauty 4 Point Eyebrow Pen Micro Ink Brow Pen Waterproof Eyebrow Pencil with Micro-Fork Tips for Daily Natural Eye Brown Makeup (1# Chestnut)
title: Lunana Beauty 4 Point Eyebrow Pen Micro Ink Brow Pen Waterproof Eyebrow Pencil with Micro-Fork Tips for Daily Natural Eye Brown Makeup (1# Chestnut)
here, brand: Lunana Beauty
at 1 title is None
tit